In [1]:
import geopandas as gp
import pandas as pd
import os
import json
import pylab as pl
from pandas.tools.plotting import scatter_matrix
import statsmodels.api as sm
import statsmodels.formula.api as smf
import datetime as dt

%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Using curl-o to download the data, move it to PUIDATA, and read it

In [2]:
#download, move data to $PUIDATA, and read data in 
!curl -O https://data.cityofnewyork.us/api/views/rgfe-8y2z/rows.csv
cmd = "mv rows.csv " + os.getenv("PUIDATA")
#the line below is to check that my string is formatted right. I should remove it to make the notebook delivery ready
print (cmd)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3110k    0 3110k    0     0  7882k      0 --:--:-- --:--:-- --:--:-- 14.3M
mv rows.csv /home/cusp/wbx200/PUIdata


In [3]:
os.system(cmd)

0

In [4]:
!curl -O http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/mn_mappluto_16v1.zip
os.system("unzip mn_mappluto_16v1.zip")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.6M  100 11.6M    0     0  26.4M      0 --:--:-- --:--:-- --:--:-- 26.8M


0

# Storing it in a subdirectory called "Manhattan" within PUIDATA.  NOTE: In Yao's code, Yao combined everything above into a function ("getpluto") that does exactly the same thing

In [5]:
#I want the files to be stored in a single directory so that the place is less messy
# so I am creating a subdir of PUIdata and putting the unzipped files there. (Not required)
cmd = "mkdir " + os.getenv("PUIDATA") + "/Manhattan"
print (cmd)
os.system(cmd)

cmd = "mv mn* MN* PLUTO* " + os.getenv("PUIDATA")+"/Manhattan"
print (cmd)
os.system(cmd)

mkdir /home/cusp/wbx200/PUIdata/Manhattan
mv mn* MN* PLUTO* /home/cusp/wbx200/PUIdata/Manhattan


0

# Setting up two dataframes from the downloaded data ("nrg" and "bsize") and plotting a scatter matrix of the energy data

In [ ]:
nrg = gp.GeoDataFrame.from_csv(os.getenv("PUIDATA") + "/Energy_and_Water_Data_Disclosure_for_Local_Law_84__2013_.csv")
# exploring the data a bit
sfig = scatter_matrix (nrg, s=300, figsize=(10, 10), diagonal='kde')

bsize = gp.GeoDataFrame.from_file(os.getenv("PUIDATA") + "/Manhattan/MNMapPLUTO.shp")

# Checking the data columns

In [ ]:
nrg.columns

In [ ]:
bsize.columns

# Dropping columns that aren't necessary 

In [ ]:
nrg.drop(['Co-reported BBL Status',
       'BBLs Co-reported',
       'Reported NYC Building Identificaiton Numbers (BINs)', 'Street Number',
       'Street Name', 'Borough', 'Zip Code',
       'DOF Benchmarking Submission Status', 
       'Weather Normalized Site EUI(kBtu/ft2)', 'Source EUI(kBtu/ft2)',
       'Weather Normalized Source EUI(kBtu/ft2)',
       'Municipally Supplied Potable Water - Indoor Intensity (gal/ft²)',
       'Automatic Water Benchmarking Eligible', 'Reported Water Method',
       'ENERGY STAR Score', 'Total GHG Emissions(MtCO2e)',
       'Direct GHG Emissions(MtCO2e)', 'Indirect GHG Emissions(MtCO2e)',
       'DOF Property Floor Area (Buildngs and Parking)(ft2)',
       'Primary Property Type - Self Selected', 'DOF Number of Buildings'], axis=1, inplace=True)

In [ ]:
bsize.drop(['APPBBL', 'APPDate', 'Address', 'AllZoning1', 'AllZoning2',
       'AreaSource', 'AssessLand', 'AssessTot', 'BldgArea', 'BldgClass',
       'BldgDepth', 'BldgFront', 'Block', 'BoroCode', 'Borough', 'BsmtCode',
       'BuiltCode', 'BuiltFAR', 'CB2010', 'CD', 'CT2010', 'ComArea', 'CommFAR',
       'CondoNo', 'Council', 'EDesigNum', 'Easements', 'ExemptLand',
       'ExemptTot', 'Ext', 'FacilFAR', 'FactryArea', 'FireComp', 'GarageArea',
       'HealthArea', 'HistDist', 'IrrLotCode', 'LandUse', 'Landmark', 'Lot',
       'LotArea', 'LotDepth', 'LotFront', 'LotType', 'LtdHeight', 'MAPPLUTO_F',
       'NumBldgs', 'NumFloors', 'OfficeArea', 'OtherArea', 'Overlay1',
       'Overlay2', 'OwnerName', 'OwnerType', 'PLUTOMapID', 'PolicePrct',
       'ProxCode', 'ResArea', 'ResidFAR', 'RetailArea', 'SHAPE_Area',
       'SHAPE_Leng', 'SPDist1', 'SPDist2', 'Sanborn', 'SanitBoro', 'SanitDist',
       'SanitSub', 'SchoolDist', 'SplitZone', 'StrgeArea', 'TaxMap',
       'Tract2010', 'Version', 'XCoord', 'YCoord',
       'YearAlter1', 'YearAlter2', 'ZMCode', 'ZipCode',
       'ZoneDist1', 'ZoneDist2', 'ZoneDist3', 'ZoneDist4', 'ZoneMap',
       'geometry'], axis=1, inplace=True)

# This is the pd.to_numeric function that Kelsey brought up for HW6 - the "coerce" function is used to convert all non-numerical data to NaN's

In [ ]:
nrg['Reported Property Floor Area (Building(s)) (ft²)'] = \
            pd.to_numeric(nrg['Reported Property Floor Area (Building(s)) (ft²)'], 

# First we rename a specific column to remove the spaces (to "BBL") and then use that as the basis for the merge between the two data sets.  Keep in mind that she mentioned three ways to do it: (HOW: inner, outer, and I forgot the other...)

In [ ]:

#renaming the quantity of interest 
#so i can refer to the column as an attribute if i want to while now i can't because the names contain spaces.
#also I want the same name for the common column BBL in both dataframes for merging
nrg.rename(columns={'NYC Borough, Block, and Lot (BBL)':'BBL'}, 
           inplace=True)
nrg.head()

In [ ]:
bblnrgdata = pd.merge(nrg, bsize, how='inner', on=['BBL'])

# She multiples area by kBtu/ft2 to get total energy per building, prints the min and max, and then plots it.

In [ ]:
## multiplying area by energy/area to get total energy per building
bblnrgdata['nrg'] = bblnrgdata[\
    'Reported Property Floor Area (Building(s)) (ft²)'].astype(float) *\
                bblnrgdata['Site EUI(kBtu/ft2)'].astype(float)
print (bblnrgdata.nrg[bblnrgdata.nrg>0].min())
print (bblnrgdata.nrg.max())
'''
indx= s1.UnitsTotal>1000
s1['UnitsTotal'][indx]=float('NaN')
indx= s1.UnitsTotal>1000
s1['UnitsTotal'][indx]=float('NaN')
'''
ax = bblnrgdata.plot(kind='scatter',x='nrg',y='UnitsTotal',
                     marker='o', figsize=(10, 10),  
                     xlim=(1000,1e11), ylim=(1,1000), fontsize=22)
yl = ax.set_ylabel("Number of Units in Building", fontsize=20)
xl = ax.set_xlabel("Energy consumption per building (kBtu)", fontsize=20)

# She specifies limits to the axes to visualize the data better.

In [ ]:
#cutting the limts in the plot to se if patterns emerge
ax = bblnrgdata.plot(kind='scatter',x='nrg',y='UnitsTotal', 
                marker='o',  figsize=(10, 10),   
                xlim=(1000,1e10), ylim=(1,1000))
yl = ax.set_ylabel("Number of Units in Building", fontsize=20)
xl = ax.set_xlabel("Energy consumption per building (kBtu)", fontsize=20)

# She specifies maxima for energy and units (arbitrarily set limits and throw out "outliers"), and plot a LOG plot in order to show a better data pattern.

In [ ]:
bblnrgdataCut = bblnrgdata[(bblnrgdata.nrg > 1000) * (bblnrgdata.UnitsTotal>=10) * 
                           (bblnrgdata.UnitsTotal<1000)]

ax = bblnrgdataCut.plot(kind='scatter', x='nrg', y='UnitsTotal', 
                   marker='o',  figsize=(16, 14), loglog=True)
yl = ax.set_ylabel("Number of Units in Building", fontsize=20)
xl = ax.set_xlabel("Energy consumption per building (kBtu)", fontsize=20)

# She defines a chi square function with three inputs: 1) the data (that we have), 2) the model (the fit lines we are going to plot), 3) and the errors (which we are going to specify)

In [ ]:
def chi2(data, model, errors = None):
    '''Calculates the chi sq given data, model and errors
    Arguments:
    data: series of datapoints (endogenous variable)
    model: series of predicted values corresponding to the observed data
    errors: serie of errors (optional). 
    If errors are not passes all errors are set to 1
    '''
    if errors is None:
        errors = np.ones_like(data)
    if data.shape == model.shape and data.shape == errors.shape:
        return (((data - model)**2) / errors**2).sum()
    else: 
        print ('''ERROR:
must pass arrays of identical dimension for data, model and (optional) error)''')
    return -1

# She specifies errors on both nrg and unit data.  NRG data error is added in quadriture (since it's from two different sources) and unit error is just square root

In [ ]:
## Assume that there is error in the reported energy. 
## but that is the product of two measured qusntities, each of which will have errors. 
## The minimum error is the squareroot of the value

#errors on the measured quantities
errorsnrg = np.sqrt((bblnrgdataCut['Reported Property Floor Area (Building(s)) (ft²)'])**2 +\
                (bblnrgdataCut['Site EUI(kBtu/ft2)']**2))

## Assume count statistics in the number of units as well
errorsunits = np.sqrt(bblnrgdataCut.UnitsTotal)

# She propogates the errors on the log plots (based off of Wiki) and specifies them within the data frames

In [ ]:
errorsInLogNrg = np.abs(errorsnrg / bblnrgdataCut.nrg / np.log(10))
errorsInLogUnits = np.abs(errorsunits / bblnrgdataCut.UnitsTotal / np.log(10))

bblnrgdataCut['errorsnrg'] =errorsInLogNrg
bblnrgdataCut['errorsunits'] = errorsInLogUnits

# She defines a line-fit function that has two inputs (x and y) and three outputs (slope, intercept of best fit, and the full model fit).  These three outputs are defined as p0, p1, and linmodel_0.  The scatter plot and linmodel_0 (the full model fit) are plotted in the figure.  The second section of code is EXACTLY the same as the previous, except the x and the y values are flipped.

In [ ]:
def fit_line1(x, y):
    """Fits a line to data properly adding the dimensions required by statsmodels
    Arguments:
        x: series of exogenous variables
        y: seried of endogenous variables
    Output:
        slope, intercept of best fit line, and the full model fit
    """
    #print x
    X = sm.add_constant(x)
    #print X
    model = sm.OLS(y, X, missing='drop') # ignores entires where x or y is NaN
    fit = model.fit()
    return fit.params[1], fit.params[0], fit # could also return stderr in each via fit.bse

p1, p0, linmodel_0 = fit_line1(np.log10(bblnrgdataCut.nrg), 
                        np.log10(bblnrgdataCut.UnitsTotal))
pl.figure(figsize=(10, 10))
pl.scatter(np.log10(bblnrgdataCut.nrg), np.log10(bblnrgdataCut.UnitsTotal))
plot(np.log10(bblnrgdataCut.nrg), linmodel_0.predict(), 'k')
yl = pl.ylabel("log 10 Number of Units in Building", fontsize=20)
xl = pl.xlabel("building log10 Energy consumption per (kBtu)", fontsize=20)
linmodel_0.summary()

In [ ]:
p1, p0, linmodel_1 = fit_line1(np.log10(bblnrgdataCut.UnitsTotal), 
                        np.log10(bblnrgdataCut.nrg))
pl.figure(figsize=(10,10))
pl.scatter(np.log10(bblnrgdataCut.UnitsTotal), np.log10(bblnrgdataCut.nrg))
plot(np.log10(bblnrgdataCut.UnitsTotal), linmodel_1.predict(), 'k')
xl = pl.xlabel("log10 Number of Units in Building", fontsize=20)
yl = pl.ylabel("building log10 Energy consumption per (kBtu)", fontsize=20)
linmodel_1.summary()

#  She recreates the plots above, but this time with error included

In [ ]:
fig = pl.figure(figsize=(18, 14))
ax1 = fig.add_subplot(221)
ax1.errorbar(np.log10(bblnrgdataCut.nrg), np.log10(bblnrgdataCut.UnitsTotal), 
                 yerr=bblnrgdataCut.errorsunits, fmt='o')
ax1.set_ylabel("log10 of Number of Units in Building", fontsize=20)
ax1.set_xlabel("building log10 of Energy  per  (kBtu)", fontsize=20)
ax1.set_title("Total units in building as function of Energy")


ax3 = fig.add_subplot(223)
ax3.errorbar(np.log10(bblnrgdataCut.UnitsTotal), np.log10(bblnrgdataCut.nrg), 
                 yerr=bblnrgdataCut.errorsnrg, fmt='o')
ax3.set_xlabel("log10 of Number of Units in Building", fontsize=20)
ax3.set_ylabel("building log10 of Energy  per  (kBtu)", fontsize=20)
ax3.set_title("Energy as function of total units in building")

ax4 = fig.add_subplot(224)
ax4.errorbar(np.log10(bblnrgdataCut.UnitsTotal), np.log10(bblnrgdataCut.nrg), 
                 yerr=bblnrgdataCut.errorsnrg, fmt='o')
ax4.set_xlabel("log10 of Number of Units in Building", fontsize=20)
ax4.set_ylabel("building log10 of Energy  per  (kBtu)", fontsize=20)
ax4.set_ylim(5,11)
ax4.set_title("Energy as function of total units in building, zoom-in")

print ("The largest error bar is for")
bblnrgdataCut[bblnrgdataCut.errorsnrg == bblnrgdataCut.errorsnrg.max()]

# She compares the chi-square values for all the different line fits that were plotted.  We are looking for the fits with the smallest chi-square values, but it varies depending on whether or not errors are involved.  She makes a point that errors MUST be included, since if we were just comparing the numbers (the residuals) there is nothing relating the two.

In [ ]:
print ("Units vs Energy residuals (no errors include): %.2f"%\
       chi2(np.log10(bblnrgdataCut.UnitsTotal), linmodel_0.predict()))
print ("Energy vs Units residuals (no errors include): %.2f"%\
        chi2(np.log10(bblnrgdataCut.nrg), linmodel_1.predict()))

print ("Units vs Energy chi square w IV error only: %.2f"%\
       chi2(np.log10(bblnrgdataCut.UnitsTotal), linmodel_0.predict(), 
            errors = bblnrgdataCut.errorsunits))
print ("Energy vs Units chi square w IV error only: %.2f"%\
        chi2(np.log10(bblnrgdataCut.nrg), linmodel_1.predict(), 
             errors = bblnrgdataCut.errorsnrg))

print ("Units vs Energy chi square: %.2f"%\
       chi2(np.log10(bblnrgdataCut.UnitsTotal), linmodel_0.predict(), 
            errors = np.sqrt(bblnrgdataCut.errorsnrg**2 + bblnrgdataCut.errorsunits**2)))
print ("Energy vs Units chi square: %.2f"%\
        chi2(np.log10(bblnrgdataCut.nrg), linmodel_1.predict(), 
             errors = np.sqrt(bblnrgdataCut.errorsnrg**2 + bblnrgdataCut.errorsunits**2)))

# She plots the second degree polynomial fit, the key here being that the formula is a function of NRG (y-intercept), units (x value), and units squared (x value squared).

In [ ]:
#I find the easiest way to use the formula package is to use a dataframe 
#with the quantities that are not linear already calculated

bblnrgdataCut['logNrg']  = np.log10(bblnrgdataCut.nrg)
bblnrgdataCut['logUnits']  = np.log10(bblnrgdataCut.UnitsTotal)

X = np.linspace(bblnrgdataCut['logUnits'].min(), bblnrgdataCut['logUnits'].max(), 1000)
curvemodel = smf.ols(formula = 'logNrg ~ logUnits + I(logUnits**2)', 
                          data = bblnrgdataCut).fit()
pl.figure(figsize=(16,14))
pl.scatter(np.log10(bblnrgdataCut.UnitsTotal), np.log10(bblnrgdataCut.nrg))
plot(X, curvemodel.predict(exog = dict(logUnits = X)), 'k')
xl = pl.xlabel("log10 Number of Units in Building", fontsize=20)
yl = pl.ylabel("building log10 Energy consumption per (kBtu)", fontsize=20)
curvemodel.summary()

# She uses the LR test to compare the linear fit and the curve fit, which she can do since they are nested models.  The corresponding LR value at a significant level of 0.05 with a chi square distribution with 1 DOF is 3.84 - if the calculated LR value is greater, we can reject the null hypothesis that the linear model is better.

In [ ]:
alpha = 0.05
print ("LR : ", -2 * (linmodel_1.llf - (curvemodel.llf)))
print ("LR from statsmodels:", curvemodel.compare_lr_test(linmodel_1))
LR = curvemodel.compare_lr_test(linmodel_1)

print ("We ", end="")
if LR[0] < 3.84: #0.05 level for 1 DOF chi sq distribution 
    print ("CANNOT") 
    
print ("reject the Null hypothesis that the restricted (linear) " + 
       "model is better than the 2nd degree polynomial fit with p-value ", end="")
print ("p < %.3f"%alpha)